# Filtering Data Loss
This notebook profiles how we loose data as we join race information and weather data.

In [14]:
import os
import pandas as pd
from datetime import datetime, timedelta

In [5]:

RAW_RACE_RESULTS_DIR = "data/race_results"

total_rows = 0

for race_file in os.listdir(RAW_RACE_RESULTS_DIR):
    filepath = os.path.join(RAW_RACE_RESULTS_DIR, race_file)
    df = pd.read_parquet(filepath)
    
    total_rows += df.shape[0]

total_raw_race_result_rows = total_rows
print(f"Total Starting Rows: {total_raw_race_result_rows:,}")

Total Starting Rows: 12,656,862


# Step 1: Extract Race Records from Race Results
In this step we extracted normalized race records from the raw race results files.  We filtered some records that did not have sufficient metadata for the downstream analysis.  This is done in the Data_Profile_Analysis.ipynb

In [9]:
RACE_RECORDS_DIR = "data/race_records"
total_rows = 0

for race_file in os.listdir(RACE_RECORDS_DIR):
    filepath = os.path.join(RACE_RECORDS_DIR, race_file)
    df = pd.read_parquet(filepath)
    
    total_rows += df.shape[0]

total_race_records_rows = total_rows
dropped_rows = total_raw_race_result_rows-total_race_records_rows
print(f"Race record rows: {total_race_records_rows:,}")
print(f" dropped {dropped_rows:,} rows, {dropped_rows/total_raw_race_result_rows*100.0:.2f}%")

Race record rows: 10,478,198
 dropped 2,178,664 rows, 17.21%


# Step 2: Filter Race Records to ones that have a non null city and state

This is done in the process_racedata.py module, and is necessary because we need the city and state to do our downstream analysis.

In [29]:
RACE_FINAL_FILE = "data/race_final/global/data.csv"
df = pd.read_csv(RACE_FINAL_FILE)

total_race_final_rows = df.shape[0]
dropped_rows = total_race_records_rows - total_race_final_rows
print(f"Final race record rows: {total_race_final_rows:,}")
print(f" dropped {dropped_rows:,} rows, {dropped_rows/total_race_records_rows*100.0:.2f}%")


Final race record rows: 7,411,671
 dropped 3,066,527 rows, 29.27%


# Step 3: Construct the featurized weather data frame
In this step we join the weather data with the race results.  This is done in the Map_Weather_Features.ipynb file, and relies on a weather forceast dataset that is built in the Map_Cities.ipynb notebook.

## Step 3(a): Filter the race records to ones that are after 2016
We only pull weather data after 2015 to simplify things, so races that are run in 2015 and before will not have enough information for our analysis

In [18]:
RACE_FINAL_FILE = "data/race_final/global/data.csv"
df = pd.read_csv(RACE_FINAL_FILE)

def parse_date_column(col:str) -> datetime:
    (month, day, year) = col.split("_")
    return datetime(int("20"+year), int(month), int(day))

df["date"] = df["date"].apply(parse_date_column)

EARLIEST_WEATHER = datetime(2015,1,1)
race_df_filtered_date = df[df["date"]> EARLIEST_WEATHER + timedelta(days=365)]

total_inrange_race_final_rows = race_df_filtered_date.shape[0]
dropped_rows = total_race_final_rows - total_inrange_race_final_rows
print(f"Total in date range race record rows: {total_inrange_race_final_rows:,}")
print(f" dropped {dropped_rows:,} rows, {dropped_rows/total_race_final_rows*100.0:.2f}%")


Total in date range race record rows: 2,170,201
 dropped 5,241,470 rows, 70.72%


## Step 3(b): Filter the race records to ones that have cities that join with the weather data
We only pulled weather data for a subset of the cities, and so we can only use race records for runners that are from those cities.

In [21]:
WEATHER_DATA = "data/weather_data.csv"

weather_data = pd.read_csv(WEATHER_DATA, parse_dates=['date'])
available_weather_cities = set(weather_data.groupby(["city","state"]).groups)

# Create a set of lowercase (city, state) tuples for fast lookup
available_weather_cities_lower = {(city.lower(), state.lower()) for (city, state) in available_weather_cities}

# Filter to only records with matching weather data
race_df_filtered_loc = race_df_filtered_date[
    race_df_filtered_date.apply(
        lambda row: (row['city'], row['state']) in available_weather_cities_lower, 
        axis=1
    )
]

unavailable_cities = race_df_filtered_date[~race_df_filtered_date.apply(
        lambda row: (row['city'], row['state']) in available_weather_cities_lower, 
        axis=1
    )].groupby(["city","state"]).groups

total_loc_race_final_rows = race_df_filtered_loc.shape[0]
dropped_rows = total_inrange_race_final_rows - total_loc_race_final_rows
print(f"Total location filtered race record rows: {total_loc_race_final_rows:,}")
print(f" dropped {dropped_rows:,} rows, {dropped_rows/total_inrange_race_final_rows*100.0:.2f}%")

Total location filtered race record rows: 358,982
 dropped 1,811,219 rows, 83.46%


In [ ]:
missing_cities = race_df_filtered_date[~race_df_filtered_date.apply(
        lambda row: (row['city'], row['state']) in available_weather_cities_lower, 
        axis=1
    )].groupby(["city","state"]).size().sort_values(ascending=False).reset_index(name='count')

missing_cities.head(20)


,city,state,count
0,cincinnati,oh,12053
1,dallas,tx,9999
2,pittsburgh,pa,9681
3,boston,ma,9136
4,atlanta,ga,8846
5,charlotte,nc,8391
6,columbus,oh,7602
7,miami,fl,7031
8,orlando,fl,6714
9,san jose,ca,6399
